In [17]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [18]:
FeatureSetB = pd.read_csv('MIMIC_Windows/FSB_1Ws.csv')
FeatureMortality = pd.read_csv('MIMIC_Windows/FSB_Mortality.csv')

In [19]:
numberOfWindows = 1
numberOfFeatures = 20

In [20]:
Subject_id = pd.DataFrame()
Subject_id = FeatureSetB.subject_id
Subject_id.drop_duplicates(keep = 'first', inplace = True)
Subject_id.reset_index(drop=True,inplace=True)
NumSubjects = len(Subject_id)

In [21]:
X = FeatureMortality['subject_id'].equals(Subject_id)

In [22]:
FeatureSet = FeatureSetB.drop(['subject_id'], axis=1)

In [23]:
FeatureSet = FeatureSet.to_numpy()

In [24]:
Labels = pd.DataFrame()
Labels['Expired'] = FeatureMortality['Expired'] 
y_values = Labels.to_numpy()
y_values = y_values.flatten()

In [25]:
nClients = NumSubjects

Testing ANN

In [26]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn import metrics
from keras.layers import Dense, Activation, Flatten
import tensorflow as tf
import sys
mod=sys.modules[__name__]

In [27]:
PR = [
      keras.metrics.AUC(name='prc',multi_label=True,curve='PR'),
    ]

In [28]:
np.random.seed(1)
tf.random.set_seed(1234)  # Fixing the seed values for reproduciblity

model = Sequential()  
model.add(Flatten(input_shape=(numberOfWindows,numberOfFeatures)))
model.add(Dense(100))
model.add(Dropout(0.2))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()  #number of parameters in dense layer is Wx + b, output=# neurons, (param = output * input + output * b)
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=PR)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 20)                0         
                                                                 
 dense_3 (Dense)             (None, 100)               2100      
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 activation_2 (Activation)   (None, 100)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
 activation_3 (Activation)   (None, 1)                

In [29]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
import numpy as np
from imblearn.under_sampling import NeighbourhoodCleaningRule
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

In [30]:
import warnings
warnings.filterwarnings("ignore")

In [31]:
acc_score = []
re_score = []
pre_score = []
f_score = []
auroc = []
auprc = []

resample = NeighbourhoodCleaningRule() 

X = FeatureSet
y = y_values

k = 5
ep = 75

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

X_test =  scaler.transform(X) 
Train_shape = X.shape

X = X.reshape(int(Train_shape[0]/numberOfWindows),numberOfWindows,numberOfFeatures)

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
# enumerate the splits and summarize the distributions

for train_ix, val_ix in skf.split(X,y):
    # select rows
    X_train, X_val = X[train_ix], X[val_ix]
    y_train, y_val = y[train_ix], y[val_ix]
    
    resample.fit_resample(X_train[:,:,0], y_train)
    X_train = X_train[resample.sample_indices_]
    y_train = y_train[resample.sample_indices_]
    
    history4 = model.fit(X_train,y_train, epochs = ep,batch_size=64)
    y_pred = model.predict(X_val)
    y_pred = list(map(lambda x: 0 if x<0.5 else 1, y_pred))
    #class_labels = np.argmax(y_pred, axis=1)
    y_actu = pd.Series(y_val)
    y_pred = pd.Series(y_pred)
     
    sensitivity = recall_score(y_actu, y_pred,pos_label = 1, average='binary')
    precision = precision_score(y_actu, y_pred,pos_label = 1, average='binary')
    f1_value = f1_score(y_actu, y_pred,pos_label = 1, average='binary')
    prc = average_precision_score(y_actu, y_pred,pos_label = 1)
    roc = roc_auc_score(y_actu, y_pred)
    accuracy = accuracy_score(y_actu, y_pred)
    
    acc_score.append(accuracy)
    re_score.append(sensitivity)
    pre_score.append(precision)
    f_score.append(f1_value)
    auroc.append(roc)
    auprc.append(prc)
   
avg_acc_score = sum(acc_score)/k
avg_recall_score = sum(re_score)/k
avg_precision_score = sum(pre_score)/k
avg_f1_score = sum(f_score)/k
avg_roc_score = sum(auroc)/k
avg_prc_score = sum(auprc)/k


sensitivity = avg_recall_score
precision = avg_precision_score
accuracy = avg_acc_score
f1_score = avg_f1_score
auroc = avg_roc_score
auprc = avg_prc_score

Epoch 1/75


2022-09-02 08:55:52.792390: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


72/72 [==============================] - 2s 15ms/step - loss: 0.3455 - prc: 0.2458
Epoch 2/75
72/72 [==============================] - 1s 12ms/step - loss: 0.2333 - prc: 0.4560
Epoch 3/75
72/72 [==============================] - 1s 12ms/step - loss: 0.2214 - prc: 0.4785
Epoch 4/75
72/72 [==============================] - 1s 12ms/step - loss: 0.2126 - prc: 0.5201
Epoch 5/75
72/72 [==============================] - 1s 12ms/step - loss: 0.2070 - prc: 0.5398
Epoch 6/75
72/72 [==============================] - 1s 12ms/step - loss: 0.2027 - prc: 0.5462
Epoch 7/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1994 - prc: 0.5562
Epoch 8/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1952 - prc: 0.5780
Epoch 9/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1952 - prc: 0.5714
Epoch 10/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1930 - prc: 0.5841
Epoch 11/75
72/72 [==============================] - 1s 12ms/step - lo

2022-09-02 08:57:00.525009: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Epoch 1/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1654 - prc: 0.6673
Epoch 2/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1572 - prc: 0.6917
Epoch 3/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1575 - prc: 0.6931
Epoch 4/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1522 - prc: 0.7141
Epoch 5/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1499 - prc: 0.7148
Epoch 6/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1505 - prc: 0.7112
Epoch 7/75
72/72 [==============================] - 1s 12ms/step - loss: 0.1459 - prc: 0.7344
Epoch 8/75
72/72 [==============================] - 1s 13ms/step - loss: 0.1463 - prc: 0.7307
Epoch 9/75
72/72 [==============================] - 1s 13ms/step - loss: 0.1466 - prc: 0.7240
Epoch 10/75
72/72 [==============================] - 1s 13ms/step - loss: 0.1455 - prc: 0.7357
Epoch 11/75
72/72 [==============================] - 1s 12m

In [32]:
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))
print('Recall of each fold - {}'.format(re_score))
print('Avg Reccall : {}'.format(avg_recall_score))
print('Precision of each fold - {}'.format(pre_score))
print('Avg Precision : {}'.format(avg_precision_score))
print('F1_Score of each fold - {}'.format(f_score))
print('Avg F1_score : {}'.format(avg_f1_score))
print('AUROC of each fold - {}'.format(auroc))
print('Avg AUROC : {}'.format(avg_roc_score))
print('AUPRC of each fold - {}'.format(auprc))
print('Avg AUPRC : {}'.format(avg_prc_score))

accuracy of each fold - [0.9064685314685315, 0.9082167832167832, 0.923951048951049, 0.9396853146853147, 0.9492563429571304]
Avg accuracy : 0.9255156042557615
Recall of each fold - [0.24271844660194175, 0.4174757281553398, 0.4519230769230769, 0.625, 0.7184466019417476]
Avg Reccall : 0.4911127707244212
Precision of each fold - [0.46296296296296297, 0.48863636363636365, 0.6103896103896104, 0.6842105263157895, 0.7184466019417476]
Avg Precision : 0.5929292130492948
F1_Score of each fold - [0.3184713375796179, 0.4502617801047121, 0.5193370165745856, 0.6532663316582915, 0.7184466019417476]
Avg F1_score : 0.531956613571791
AUROC of each fold - 0.7298901435828563
Avg AUROC : 0.7298901435828563
AUPRC of each fold - 0.35318540112438473
Avg AUPRC : 0.35318540112438473


In [33]:
import csv
import os.path
from datetime import datetime

csv_columns = ['model-type','precision','sensitivity','f1-score','accuracy','AUROC','AUPRC','NumberOfWindows','Epochs']
dict_data = [{'model-type':'ANN', 'precision': precision,'sensitivity': sensitivity,'f1-score': f1_score,'accuracy': accuracy,'NumberOfWindows':numberOfWindows,"Epochs":ep,'AUROC':auroc,'AUPRC':auprc}]
metric_file = "Results/Results_ANN.csv"

file_exists = os.path.isfile(metric_file)
try:
    with open(metric_file, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        if not file_exists:
            writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
except IOError:
    print("I/O error")